In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [13]:
names = ["class", "message"]
train_file = pd.read_csv(train_file_path, sep='\t', names=names)
test_file = pd.read_csv(test_file_path, sep='\t', names=names)

In [14]:
train_file.loc[train_file['class'] == 'ham', 'class'] = 0
train_file.loc[train_file['class'] == 'spam', 'class'] = 1

test_file.loc[test_file['class'] == 'ham', 'class'] = 0
test_file.loc[test_file['class'] == 'spam', 'class'] = 1

train_message = train_file['message'].values.tolist()
train_label = train_file['class'].values
test_message = test_file['message'].values.tolist()
test_label = test_file['class'].values

train_label = train_label.astype('int32')
test_label = test_label.astype('int32')

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Step 1: Initialize the tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")  # You can adjust num_words

# Step 2: Fit the tokenizer on the training data
tokenizer.fit_on_texts(train_message)

In [16]:
# Step 3: Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_message)
test_sequences = tokenizer.texts_to_sequences(test_message)

In [ ]:
train_sequences[0]  #contains the tokenized integer sequence representation of the first message from train_message[0]

In [8]:
# print(tokenizer.word_index)

# inv_vocab = {v: k for k, v in tokenizer.word_index.items()}
# decoded = [inv_vocab.get(i, "<OOV>") for i in train_sequences[0]]
# print(decoded)

In [20]:
# Calculate VOCAB_SIZE including the OOV token
# The tokenizer's word_index does not include the OOV token by default.
# Since index 0 is reserved for the OOV token, the total vocabulary size
# for the embedding layer is the number of unique words + 1 for the OOV token.

VOCAB_SIZE = len(tokenizer.word_index) + 1
max_len = max(len(seq) for seq in train_sequences)

In [21]:
# Step 4: Pad the sequences (to ensure all have same length)
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

In [22]:
from tensorflow.keras.callbacks import EarlyStopping

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 100),
    # tf.keras.layers.LSTM(100),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(train_padded, train_label, epochs=30, validation_data=(test_padded, test_label), callbacks=[early_stop])


Epoch 1/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - acc: 0.8664 - loss: 0.3188 - val_acc: 0.9763 - val_loss: 0.0966
Epoch 2/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - acc: 0.9847 - loss: 0.0592 - val_acc: 0.9820 - val_loss: 0.0516
Epoch 3/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - acc: 0.9930 - loss: 0.0252 - val_acc: 0.9864 - val_loss: 0.0406
Epoch 4/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - acc: 0.9975 - loss: 0.0121 - val_acc: 0.9871 - val_loss: 0.0402
Epoch 5/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.9993 - loss: 0.0068 - val_acc: 0.9864 - val_loss: 0.0401
Epoch 6/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - acc: 0.9997 - loss: 0.0044 - val_acc: 0.9892 - val_loss: 0.0347
Epoch 7/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.9998 - loss: 0.0033 - val_acc: 0.9871 - val_loss: 0.0385
Epoch 8/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.9996 - loss: 0.0032 - val_acc: 0.9871 - val_loss: 0.0386


## ✅ Why the Simple Model Performs Better

Your simple model:

```python
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 100),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
```
- Uses an Embedding layer to convert words into vectors.

- Applies Flatten, which ignores word order and treats the entire sequence as a bag of embeddings.

- This is often enough for tasks like spam detection, where certain keywords ("free", "win", "offer") strongly indicate spam.

- Has fewer parameters, making it less prone to overfitting and fast to train.

- Performs well on small and simple datasets.

📈 As a result, this model can easily reach very high accuracy (even close to 100%) when the classification task is largely based on presence of specific words.

## ❌ Why the LSTM Model Underperforms
Your LSTM model:

```python
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 100),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
```
- Introduces a lot more parameters, making it more complex and harder to train, especially on small datasets.

- Tries to learn word order and contextual relationships, which may not be crucial for spam classification.

- Requires more data and longer training to show its strengths.

- Is more prone to underfitting or overfitting without proper regularization.

⚠️ On small datasets like SMS spam, the LSTM might not have enough signal to learn useful patterns and may struggle to match the performance of simpler models.

In [23]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

    sequence = tokenizer.texts_to_sequences([pred_text])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')

    prob = model.predict(padded)[0][0]  # Get the probability from the prediction output

    label = 'spam' if prob > 0.5 else 'ham'
    return [prob, label]

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
[np.float32(0.58725935), 'spam']


In [24]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[np.float32(0.0012677497), 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[np.float32(0.58725935), 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
[np.float32(7.012616e-05), 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[np.float32(0.99209875), 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[np.float32(0.99884593), 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[np.float32(0.00042806624), 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[np.float32(0.0010556177), 'ham']
You passed the challenge. Great job!
